<a href="https://colab.research.google.com/github/RAHULRNAIR2000/RAG-chatbot--poc/blob/main/ragchatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()


Saving f1.pdf to f1.pdf


In [4]:
import pypdf
from tqdm import tqdm

file_path = "f1.pdf"  # Fixed the string literal
pdf_data = pypdf.PdfReader(file_path)

pdf_text = ""
for page in tqdm(pdf_data.pages):
    pdf_text += f"{page.extract_text()}\n"

print(pdf_text)


100%|██████████| 107/107 [00:10<00:00,  9.84it/s]

 
  
2022 Formula 1 Sporting Regulations 1/107 30 September 2022 
©2022 Fédération Internationale de l’Automobile  Issue 2 
 2023 FORMULA ONE SPORTING REGULATIONS 
PUBLISHED ON 30 SEPTEMBER 2022 
ISSUE 2 
Convention: 
Black text: As approved for 2022 by the WMSC up to and including 19/7/22 
Pink text: Changes for 2023 approved by the WMSC up to and including 19/7/22 
Pink highlighted text: Changes for 2023 approved by the WMSC on 27/9/22 
 
ART CONTENTS PAGE 
1 REGULATIONS 2 
2 GENERAL UNDERTAKING 2 
3 GENERAL CONDITIONS 2 
4 LICENCES 3 
5 CHAMPIONSHIP COMPETITIONS 3 
6 WORLD CHAMPIONSHIP 4 
7 DEAD HEAT 6 
8 COMPETITORS APPLICATIONS 6 
9 CAR LIVERY 7 
10 TRACK RUNNING TIME OUTSIDE A 
COMPETITION 8 
11 PROMOTER 13 
12 ORGANISATION OF A COMPETITION 13 
13 INSURANCE 13 
14 FIA DELEGATES 14 
15 OFFICIALS 14 
16 INSTRUCTIONS AND COMMUNICATIONS  
 TO COMPETITORS 15 
17 PROTESTS AND APPEALS 15 
18 SANCTIONS 16 
19 PRESS CONFERENCES, MEDIA 
OPPORTUNITIES, DRIVERS 
 PARADE AND NATIONAL ANTHEM 1

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=256,
    chunk_overlap=50,
    length_function=len,
    is_separator_regex=False,
)

recursive_chunks = text_splitter.create_documents([pdf_text])

In [14]:
pip install chromadb


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.5/615.5 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 6.4 MB/s eta 0:00:00


In [15]:
import chromadb
COLLECTION_NAME = "text-rag"

client = chromadb.PersistentClient()
collection = client.get_or_create_collection(COLLECTION_NAME, metadata={"hnsw:space": "cosine"})
collection.count()

0

In [23]:
from sentence_transformers import SentenceTransformer

# Load a Hugging Face embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')  # Example: A lightweight model

# Create embeddings for the documents
document_embeddings = model.encode(
    [doc.page_content for doc in recursive_chunks],  # Input text
    show_progress_bar=True
)

Batches:   0%|          | 0/47 [00:00<?, ?it/s]

In [24]:
collection.add(
    ids=[str(i) for i in range(1, len(recursive_chunks) + 1)],
    documents=[doc.page_content for doc in recursive_chunks],  # Store raw text for reference
    embeddings=document_embeddings  # Store embeddings for vector search
)


In [25]:
# 5. Confirm the number of documents in the collection
print(f"Number of documents in the collection: {collection.count()}")

Number of documents in the collection: 1478


In [33]:
from sentence_transformers import SentenceTransformer

# Load the same embedding model used for documents
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# User's question
question = "Who has the authority to report on-track incidents or suspected breaches of the Sporting Regulations or the Code to the stewards?"

# Convert the question to a vector
question_vector = embedding_model.encode([question])[0]  # Single vector for the query

# Query ChromaDB for similar documents
results = collection.query(
    query_embeddings=[question_vector],
    n_results=5,  # Number of relevant documents to retrieve
    include=['documents', 'distances']  # Return documents and similarity scores
)

# Extract the top contexts
retrieved_contexts = results['documents'][0]
print("Retrieved Contexts:", retrieved_contexts)


Retrieved Contexts: ['54.1 The Race Director may report any on -track incident or suspected breach of these \nSporting Regulations or the Code (an “Incident”) to the stewards. After review it shall be', 'such incident will normally be investigated after the relevant session. \nb) Where appropriate, regard will also be given to the provisions of Article 18.1. \n37.5 Any driver taking part in any practice session who, in the opinion of the stewards, stops', 'f) Any decision taken by the stewards under Article 4.2. \n \n18) SANCTIONS \n18.1 The stewards may impose the penalties specifically set out in these Sporting Regulations \nin addition to or instead of any other penalties available to them under the Code.', 'at the discretion of the stewards to decide whether or not to proceed with an \ninvestigation. \n The stewards may also investigate an Incident noted by themselves.', '54.2 a) It shall be at the discretion of the stewards to decide if any driver involved in an \nIncident should 

In [34]:
from transformers import pipeline

# Load a generative model for answering questions
qa_model = pipeline("text2text-generation", model="google/flan-t5-large")

# Combine contexts
combined_context = " ".join(retrieved_contexts)

# Ask the question with the combined context
response = qa_model(f"Question: {question} Context: {combined_context}")

print("Answer:", response[0]['generated_text'])


Answer: The Race Director
